In [2]:
from config_file import config
from dataset import RetroGamesHelper
import pandas as pd
from pathlib import Path
from PIL import Image
from datasets import load_dataset

e:\PythonProjects\Scraping Dataset\generating-annotations\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [92]:
def sanitize_filename(name):
    special = "\\/:*?\"<>|+'"
    new_name = "".join(c if c not in special else "_" for c in name)

    if new_name[-1] in " .":
        new_name = new_name[:-1] + "_"

    return new_name

In [114]:
dataset_path = config.DATASET_PATH
d = load_dataset(str(dataset_path))

Generating train split: 16828 examples [00:00, 256226.71 examples/s]
Generating test split: 6516 examples [00:00, 411976.14 examples/s]


In [118]:
load_dataset(str(dataset_path), "games_info")[0]

KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['train']"

In [ ]:
test_df = pd.read_csv(config.DATA_PATH / "test_games_genres_igdb.csv")
train_df = pd.read_csv(config.DATA_PATH / "train_games_genres_igdb.csv")
# game_titles = pd.DataFrame(pd.concat(
#     [test_df["title"],
#     train_df["title"]]
# ))
# game_titles["game_folder"] = game_titles["title"].apply(sanitize_filename)
# game_titles.to_csv(config.DATA_PATH / "name_transform.csv", index=False)

In [96]:
name_transform = pd.read_csv(config.DATASET_PATH / "name_transform.csv")

def transform_name(name: str, name_transform):
    result = name_transform.loc[name_transform["title"] == name]["game_folder"]
    return result.to_list()[0]

name_transform

,title,game_folder
0,Bayonetta,Bayonetta
1,Call of Juarez: Bound In Blood,Call of Juarez_ Bound In Blood
2,Demon's Souls,Demon_s Souls
3,F.E.A.R.,F.E.A.R_
4,Kingdom Hearts II Final Mix,Kingdom Hearts II Final Mix
...,...,...
59,Tomb Raider II,Tomb Raider II
60,Tomb Raider: Anniversary,Tomb Raider_ Anniversary
61,Uncharted: Drake's Fortune,Uncharted_ Drake_s Fortune
62,Warcraft III: Reign of Chaos,Warcraft III_ Reign of Chaos


In [97]:
transform_name("F.E.A.R.", name_transform)

'F.E.A.R_'

In [ ]:
test_df["split"] = "test"
train_df["split"] = "train"
games_info = pd.concat([train_df, test_df])


def get_game_frames_count(game: str):
    item = games_info.loc[games_info["title"] == game]
    game_split = item["split"].to_list()[0]
    game_dir_path = dataset_path / game_split / transform_name(game, name_transform)
    frames_count = len(list(game_dir_path.glob('**/*.png')))
    return frames_count

games_info["frames_count"] = games_info["title"].apply(get_game_frames_count)
# games_info
# games_info.to_csv(config.DATASET_PATH / "games_info.csv", index=False)

In [ ]:
genres_distribution = {}

for index, row in games_info:
    